In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import re
import os
from bs4 import BeautifulSoup
import requests
import csv
import itertools
from tqdm import tqdm
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
def get_links(tag,page_limit):
    hrefs=[]
    for page in range(1,page_limit):
        req=requests.get(url='https://stackoverflow.com/questions/tagged/{}?tab=newest&page={}&pagesize=15'.format(tag,page))
        soup=BeautifulSoup(req.text,"html.parser")
        for i in soup.find_all("a",class_="s-link",href=True):
            hrefs.append(i['href'])
            
    filtered_links = [link for link in hrefs if link != "javascript:void(0)" and not link.startswith("https")]
    prefix='https://stackoverflow.com'
    final_links=[]
    for h in filtered_links:
        if 'https' not in h:
            m=prefix+h
            final_links.append(m)
            
    return final_links

In [3]:
def questions_and_anwers(links):
    questions_json=[]
    margin=1e-7
    for link in tqdm(links,total=len(links)):
        question_dict = {
                'question': None,
                'answers': None,
                'votes':None
            }


        response = requests.get(url=link)
        soup = BeautifulSoup(response.text,"html.parser")


        reply_answers=[]
        answer_votes=[]
        marg_votes=[]
        question_elements = soup.find_all('div',{'class':'question js-question'})
        for question in question_elements:
            post_body = question.find(class_="s-prose js-post-body")
            asked_question=re.sub(r'<[^>]+>', '', str(post_body))


            
            question_dict['question']=re.sub(r'\s+', ' ', asked_question).strip()

        answer_elements = soup.find_all('div',{'id':'answers'})
        for answer in answer_elements:
            post_body = answer.find_all(class_="s-prose js-post-body")
            vote_elem=answer.find_all(class_='js-vote-count')
           
            for elem in post_body:
                reply_answer=re.sub(r'<[^>]+>', '', str(elem))

                local_answers=[]
                local_answers.append( re.sub(r'\s+', ' ', reply_answer).strip())
                reply_answers.append(local_answers)
            for elem in vote_elem:
                votes=elem.get_text()

                answer_votes.append( re.sub(r'\s+', ' ', votes).strip())
                int_votes=[eval(i) for i in answer_votes]
                marg_votes=[margin+i for i in int_votes]
        question_dict['answers']=reply_answers
        question_dict['votes']=marg_votes
        questions_json.append(question_dict)
    return questions_json

In [4]:
tags1=['nvidia','nvidia-jetson','nvidia-jetson-nano','nvidia-docker','nvidia-digits','nvidia-deepstream','nvidia-smi','nvidia-titan','nvidia-flex','nvidia-hpc-compilers','nvidia-isaac','nvidia-jetpack-sdk','nvidia-sass','nvidia-modulus']
topic_pages1=[75,9,7,7,3,3,2,2,2,2,2,2,2,2]

In [5]:
tags2=['pycuda','theano-cuda','cuda-gdb','jcuda','cuda-streams','cudafy.net','cuda-context','cuda-driver','cuda-arrays','managed-cuda','cuda-graphs','cuda.net','cuda-events','cudamat','cuda-uva','scikit-cuda','cudatoolkit','cuda-profiling','cuda-jit-cache','cuda-wmma','triton','tritonserver']
topic_pages2=[10,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]

In [6]:
tags=tags1+tags2
topic_pages=topic_pages1+topic_pages2

In [7]:
cuda_page_lim=287

In [8]:
for i in range(0,len(tags)):
    links=get_links(tags[i],topic_pages[i])
    q_and_a=questions_and_anwers(links)
    with open('{}.json'.format(tags[i]), 'w') as json_file:
       json.dump(q_and_a, json_file, indent=4)

100%|██████████| 398/398 [35:03<00:00,  5.28s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 19/19 [04:20<00:00, 13.73s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
